In [2]:
import pandas as pd

In [4]:
df=pd.read_csv("players_20.csv")

In [7]:
df.describe()

,sofifa_id,age,height_cm,weight_kg,overall,potential,value_eur,wage_eur,international_reputation,weak_foot,...,mentality_penalties,mentality_composure,defending_marking,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes
count,18278.000000,18278.000000,18278.000000,18278.000000,18278.000000,18278.000000,1.827800e+04,18278.000000,18278.000000,18278.000000,...,18278.000000,18278.000000,18278.000000,18278.000000,18278.000000,18278.000000,18278.000000,18278.000000,18278.000000,18278.000000
mean,219738.864482,25.283291,181.362184,75.276343,66.244994,71.546887,2.484038e+06,9456.942773,1.103184,2.944250,...,48.383357,58.528778,46.848889,47.640333,45.606631,16.572765,16.354853,16.212934,16.368038,16.709924
std,27960.200461,4.656964,6.756961,7.047744,6.949953,6.139669,5.585481e+06,21351.714095,0.378861,0.664656,...,15.708099,11.880840,20.091287,21.585641,21.217734,17.738069,16.996925,16.613665,17.136497,18.038125
min,768.000000,16.000000,156.000000,50.000000,48.000000,49.000000,0.000000e+00,0.000000,1.000000,1.000000,...,7.000000,12.000000,1.000000,5.000000,3.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,204445.500000,22.000000,177.000000,70.000000,62.000000,67.000000,3.250000e+05,1000.000000,1.000000,3.000000,...,39.000000,51.000000,29.000000,27.000000,24.000000,8.000000,8.000000,8.000000,8.000000,8.000000
50%,226165.000000,25.000000,181.000000,75.000000,66.000000,71.000000,7.000000e+05,3000.000000,1.000000,3.000000,...,49.000000,60.000000,52.000000,55.000000,52.000000,11.000000,11.000000,11.000000,11.000000,11.000000
75%,240795.750000,29.000000,186.000000,80.000000,71.000000,75.000000,2.100000e+06,8000.000000,1.000000,3.000000,...,60.000000,67.000000,64.000000,66.000000,64.000000,14.000000,14.000000,14.000000,14.000000,14.000000
max,252905.000000,42.000000,205.000000,110.000000,94.000000,95.000000,1.055000e+08,565000.000000,5.000000,5.000000,...,92.000000,96.000000,94.000000,92.000000,90.000000,90.000000,92.000000,93.000000,91.000000,92.000000


In [12]:
df.columns.values

array(['sofifa_id', 'player_url', 'short_name', 'long_name', 'age', 'dob',
       'height_cm', 'weight_kg', 'nationality', 'club', 'overall',
       'potential', 'value_eur', 'wage_eur', 'player_positions',
       'preferred_foot', 'international_reputation', 'weak_foot',
       'skill_moves', 'work_rate', 'body_type', 'real_face',
       'release_clause_eur', 'player_tags', 'team_position',
       'team_jersey_number', 'loaned_from', 'joined',
       'contract_valid_until', 'nation_position', 'nation_jersey_number',
       'pace', 'shooting', 'passing', 'dribbling', 'defending', 'physic',
       'gk_diving', 'gk_handling', 'gk_kicking', 'gk_reflexes',
       'gk_speed', 'gk_positioning', 'player_traits',
       'attacking_crossing', 'attacking_finishing',
       'attacking_heading_accuracy', 'attacking_short_passing',
       'attacking_volleys', 'skill_dribbling', 'skill_curve',
       'skill_fk_accuracy', 'skill_long_passing', 'skill_ball_control',
       'movement_acceleration', 'mo

In [17]:
df1 = df[['short_name', 'wage_eur', 'pace', 'shooting', 'passing', 'dribbling', 'defending', 'physic']].copy()

In [22]:
clean = df1.dropna()

In [23]:
clean.head()

,short_name,wage_eur,pace,shooting,passing,dribbling,defending,physic
0,L. Messi,565000,87.0,92.0,92.0,96.0,39.0,66.0
1,Cristiano Ronaldo,405000,90.0,93.0,82.0,89.0,35.0,78.0
2,Neymar Jr,290000,91.0,85.0,87.0,95.0,32.0,58.0
4,E. Hazard,470000,91.0,83.0,86.0,94.0,35.0,66.0
5,K. De Bruyne,370000,76.0,86.0,92.0,86.0,61.0,78.0


In [24]:
clean.to_csv('clean.csv')

In [25]:
clean.short_name.count()

16242

In [28]:
len(clean.short_name.value_counts())

15481

In [36]:
short_list = clean.groupby('short_name').mean().sort_values('wage_eur',ascending=False)[:200]

In [37]:
short_list

,wage_eur,pace,shooting,passing,dribbling,defending,physic
short_name,,,,,,,
L. Messi,565000.0,87.0,92.0,92.0,96.0,39.0,66.0
E. Hazard,470000.0,91.0,83.0,86.0,94.0,35.0,66.0
Cristiano Ronaldo,405000.0,90.0,93.0,82.0,89.0,35.0,78.0
A. Griezmann,370000.0,81.0,86.0,84.0,89.0,57.0,72.0
K. De Bruyne,370000.0,76.0,86.0,92.0,86.0,61.0,78.0
...,...,...,...,...,...,...,...
R. Bentancur,86000.0,70.0,58.0,78.0,81.0,70.0,74.0
C. Aránguiz,86000.0,63.0,72.0,82.0,81.0,74.0,73.0
F. Valverde,86000.0,68.0,69.0,73.0,73.0,66.0,67.0


In [38]:
short_list.to_csv('short_list.csv')